# Evaluation of the processed amplitude *using raw data*

This processor uses the averaged data of the amplitude analyser rather than the approximated hyoerbolic function.

# Setup

In [73]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
from colorama import Fore, Back, Style
import plotting_util
from reportlab.lib import pagesizes
from reportlab.lib.units import mm
from reportlab.lib import pagesizes
A6_LS = pagesizes.landscape(pagesizes.A6)
PAGE_SMALL = (160 * mm, 80 * mm)

import configuration as cfg


In [74]:
# Import serialized symmetrical list to file
def import_list_binary(filename: str):
    with open(filename + '.dat', 'rb') as fd:
        list = pickle.load(fd)
    return list


# left channel, part a and b
# right channel
avg_la = import_list_binary("data/hyperbolic_raw_data_la")
avg_lb = import_list_binary("data/hyperbolic_raw_data_lb")
avg_ra = import_list_binary("data/hyperbolic_raw_data_ra")
avg_rb = import_list_binary("data/hyperbolic_raw_data_rb")


# Umrechnung von Amplitude auf Schallpegel (RMS) \[dB\]

Pseudocode Umrechnungsfunktionen:
```python
# a: Amplitude
amp_to_db(a: float) = 20 * log10(a)
amp_to_rms(a: float) = amp_to_db(sqrt((a * a) / 2))
```

In [75]:
def __amp_to_db(amplitude):
    """
    IMPORTANT: Returns a numpy array
    """
    return 20 * np.log10(amplitude)

def amp_to_rms(amplitude):
    amplitude = np.array(amplitude)
    return __amp_to_db(np.sqrt((np.multiply(amplitude, amplitude)) / 2))

## Three point RMS approach

~~**CAUTION:** This approach does only work with RMS values based on a hypberolic amplitude function. For discrete (raw) data, this approach returns arbitrary results.~~

*Correction:* It works nevertheless. Only difference: the velocity is only calculated using two datapoints. However, this may not be that bad, because the Gaussian filter averaged extreme deviations of the raw input data.
Currently, the points at $1/4$'ths and $2/4$'ths are used to calculate the average speed in the time frame with size ```time_increment```.    <span style="color:red">!!NOCHMAL CODE DURCHLESEN UND SCHAUEN, OB DAS RICHTIG IST, WAS ICH HIER BEHAUPTET HABE!!</span>

Because of the hyperbolic properties of the function, an arbitrary time / loudness frame can be chosen; regardless of the chosen frame, the calculated velocity is always constant.

In [76]:
# plotting_util.plot_amplitude_data(
#     "Amplitude to RMS conversion", "Hyperbola (optimal_a)", cfg.averaging_resolution, data1=symm_array, axis2_name="RMS [dB]", data2=amp_to_rms(symm_array), graph_on_same_axis=True)

# rms_curve = amp_to_rms(symm_array)

rms_curve_la = amp_to_rms(avg_la)
rms_curve_lb = amp_to_rms(avg_lb)
rms_curve_ra = amp_to_rms(avg_ra)
rms_curve_rb = amp_to_rms(avg_rb)

amt_print_vals = 3 # minimum of 3

def three_point_rms(rms_curve):
    num_samples = len(rms_curve)
    sample_increment = int(num_samples / amt_print_vals)
    time_increment = sample_increment / cfg.averaging_resolution
    rms_vals = [(rms_curve[sample_increment * (i + 1) - 1]) for i in range(amt_print_vals)]
    print(f"time increment: {str(time_increment).replace('.', ',')} s")
    for rms_value in rms_vals:
        print(str(rms_value).replace('.', ','), end='\t')
    print('\n')
    return (rms_vals, time_increment)

rms_vals_la, time_increment_la = three_point_rms(rms_curve_la)
rms_vals_lb, time_increment_lb = three_point_rms(rms_curve_lb)
rms_vals_ra, time_increment_ra = three_point_rms(rms_curve_ra)
rms_vals_rb, time_increment_rb = three_point_rms(rms_curve_rb)

time increment: 0,5833333333333334 s
-17,739260490927457	-20,66006433080976	-21,711600998490916	

time increment: 0,8333333333333334 s
-19,03715301887974	-23,592560217067877	-26,79441863981279	

time increment: 0,4166666666666667 s
-18,314302889558824	-19,69530430609588	-20,796243424917776	

time increment: 1,0 s
-21,748063142226265	-24,468159786925273	-26,528946580436337	



# Pythagoras correction

<span style="color:red; font-size:30pt;">fehlt momentan noch</span>

# Weitere Korrektur des Codes (ausstehend):

Einzelne Messpunkte erst ab **Wendepunkt** verwenden

# Speed calculation

Using the data yielded by the amplitude to RMS converter, the speed of the passing vehicle is approximated.

Since the RMS curve is derived from a hyperbola, no extra precision is gained by calculating the speed at multiple points in time. However, due to the fact that the hyperbola is only an approximation of the actual amplitude curve, especially near $x = 0$, a value further from $0$ is picked (the hyperbola approaches a value of $\infty$ near $0$, which cannot be used in this formula).

See excel sheet "AmplitudeTest.xlsx" for empirical proof that speeds calculated with $x = 0$ including, differ from all other values.

In [77]:
r1 = 2.5 # radius_of_recording
L1 = 0 # maximum loudness

def radius2_from_3_known(radius1, Loudness1, Loudness2): # -> r2
    """
    Source of formula: http://www.sengpielaudio.com/Rechner-entfernung.htm
    """
    return radius1 * (10 ** (abs(Loudness1 - Loudness2) / 20))

def calc_speed_from_hypberbolic_input(rms_vals, time_increment):
    L2 = rms_vals[0]
    L3 = rms_vals[1]

    r2 = radius2_from_3_known(r1, L1, L2)
    r3 = radius2_from_3_known(r1, L1, L3)

    v2_3 = (r3 - r2) / time_increment
    return v2_3

v_la = calc_speed_from_hypberbolic_input(rms_vals_la, time_increment_la)
v_lb = calc_speed_from_hypberbolic_input(rms_vals_lb, time_increment_lb)
v_ra = calc_speed_from_hypberbolic_input(rms_vals_ra, time_increment_ra)
v_rb = calc_speed_from_hypberbolic_input(rms_vals_rb, time_increment_rb)

print(Fore.BLACK + Back.CYAN + f"Approximated speed LA: {v_la} [m/s] = {v_la * 3.6} [km/h]" + Style.RESET_ALL)
print(Fore.BLACK + Back.CYAN + f"Approximated speed LB: {v_lb} [m/s] = {v_lb * 3.6} [km/h]" + Style.RESET_ALL)
print(Fore.BLACK + Back.CYAN + f"Approximated speed RA: {v_ra} [m/s] = {v_ra * 3.6} [km/h]" + Style.RESET_ALL)
print(Fore.BLACK + Back.CYAN + f"Approximated speed RB: {v_rb} [m/s] = {v_rb * 3.6} [km/h]" + Style.RESET_ALL)
v_hyperbola_avg = (v_la + v_lb + v_ra + v_rb) / 4
print(Fore.BLACK + Back.YELLOW +
      f"AVERAGED SPEED: {v_hyperbola_avg} [m/s] = {v_hyperbola_avg * 3.6} [km/h]" + Style.RESET_ALL)


Approximated speed LA: 13.20500895069943 [m/s] = 47.53803222251795 [km/h]
Approximated speed LB: 18.515821307682497 [m/s] = 66.65695670765699 [km/h]
Approximated speed RA: 8.51586149931339 [m/s] = 30.657101397528205 [km/h]
Approximated speed RB: 11.243170929146874 [m/s] = 40.47541534492875 [km/h]
AVERAGED SPEED: 12.869965671710549 [m/s] = 46.331876418157975 [km/h]


Multiple speed calculations with averaging

In [78]:
r1 = 2.5  # radius_of_recording
def calc_speed_from_raw_input(rms_curve):
    L1 = rms_curve[0]  # maximum loudness

    # velocity calculation for all points in the RMS list
    r_prev = r1
    v_list = np.empty(len(rms_curve))
    for L_x in rms_curve:
        r_x = radius2_from_3_known(r1, L1, L_x)
        # naming explanation: v_(previous radius)_(radius x)
        # v = s/t  ;  t = 1/res
        v_prev_x = (r_x - r_prev) * cfg.averaging_resolution
        # print(v_prev_x)
        v_list = np.append(v_list, v_prev_x)
        r_prev = r_x
    
    v_min = np.min(v_list)
    v_avg = np.mean(v_list)
    v_median = np.median(v_list)
    v_max = np.max(v_list)

    return (v_min, v_avg, v_median, v_max)


v_raw_min_avg = 0
v_raw_avg_avg = 0
v_raw_median_avg = 0
v_raw_max_avg = 0
rms_curves_list = [rms_curve_la, rms_curve_lb, rms_curve_ra, rms_curve_rb]
names_list = ["LA", "LB", "RA", "RB"]
print("Values in [m/s] \u2259 [km/h]")
print(" min | avg | median | max")
for i in range(4):
    v_min, v_avg, v_median, v_max = calc_speed_from_raw_input(rms_curves_list[i])
    print(Fore.BLACK + Back.CYAN +
          f"Approximated speed {names_list[i]}: {v_min:.1f} \u2259 {v_min*3.6:.1f} \t| \
    {v_avg:.1f} \u2259 {v_avg*3.6:.1f} \t| \
    {v_median:.1f} \u2259 {v_median*3.6:.1f} \t| \
    {v_max:.1f} \u2259 {v_max*3.6:.1f} \t" + Style.RESET_ALL)

    v_raw_min_avg += v_min
    v_raw_avg_avg += v_avg
    v_raw_median_avg += v_median
    v_raw_max_avg += v_max

v_raw_min_avg /= 4
v_raw_avg_avg /= 4
v_raw_median_avg /= 4
v_raw_max_avg /= 4




print(Fore.BLACK + Back.CYAN +
      f"""Mininmum: {v_min} [m/s] = {v_min * 3.6} [km/h]
      Average: {v_avg} [m/s] = {v_avg * 3.6} [km/h]
      Median: {v_median} [m/s] = {v_median * 3.6} [km/h]
      Maximum: {v_max} [m/s] = {v_max * 3.6} [km/h]""" + Style.RESET_ALL)


Values in [m/s] ≙ [km/h]
 min | avg | median | max
Approximated speed LA: 0.0 ≙ 0.0 	|     0.6 ≙ 2.1 	|     0.2 ≙ 0.7 	|     2.1 ≙ 7.6 	
Approximated speed LB: 0.0 ≙ 0.0 	|     1.2 ≙ 4.3 	|     0.7 ≙ 2.4 	|     4.5 ≙ 16.2 	
Approximated speed RA: 0.0 ≙ 0.0 	|     0.5 ≙ 1.9 	|     0.2 ≙ 0.7 	|     1.3 ≙ 4.6 	
Approximated speed RB: 0.0 ≙ 0.0 	|     0.9 ≙ 3.3 	|     0.6 ≙ 2.3 	|     2.9 ≙ 10.5 	
Mininmum: 0.0 [m/s] = 0.0 [km/h]
      Average: 0.92726637499708 [m/s] = 3.338158949989488 [km/h]
      Median: 0.6289598725643017 [m/s] = 2.264255541231486 [km/h]
      Maximum: 2.9183619715848614 [m/s] = 10.506103097705502 [km/h]
